In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
sys.argv=[''] 
del sys

In [3]:
cd "/root/SymmNeRF-baseline/code"

/root/SymmNeRF-baseline/code


### Test out the general code implementation. 

In [4]:
from opt import config_parser

parser = config_parser()
args = parser.parse_args()

args.num_local_layers = 2
args.local_rank = 1
args.no_load_opt = True
args.no_load_scheduler = True
args.local_feature_ch = 1024

det = args.det
lindisp = args.lindisp

device = f'cuda:{args.local_rank}'

In [5]:
from datasets import dataset_dict, create_training_dataset
from torch.utils.data import DataLoader 

from model import model_dict  
from model.sample_ray import RaySampler 
from model.render_ray import render_rays

from utils_lab_fct import * 

import setproctitle
import torch 
import numpy as np 
import tqdm 
import cv2
from einops import rearrange 
import matplotlib.pyplot as plt
from time import time
import multiprocessing as mp

setproctitle.setproctitle('[Gaetan - RayTransformer test.]')

#### 0.Build a data batch.

In [6]:
bs = 2

train_dataset, train_sampler = create_training_dataset(args)
train_loader = DataLoader(train_dataset,batch_size = bs , sampler = train_sampler)

it = iter(train_loader)
train_batch = next(it)

[Info] Training dataset: srns_dataset
[Info] Loading SRNs dataset: /data/srn_cars/cars_train


In [7]:
# RaySampler class instantiation.
ray_sampler = RaySampler(train_batch) 

# Number of rays and number of points sampled alogn each ray.
nb_rays = 128
nb_sampled_points_on_rays = 128

# Build up a batch of ray. 
ray_batch = ray_sampler.random_sample(nb_rays,use_bbox = True,device = device)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


#### 1. Instantiate a model, without any weights loading. 

In [8]:
args.cosine_mod ={"use":True,
                   'type_mod':'local_feature_mod',
                   "learn_through_hypernetwork":True,
                   "G": 64}
args.use_ray_transformer = True
model = model_dict['hypernerf_symm_local'](args,ckpts_folder = "")

[Info] No ckpts found, training from scratch...


In [ ]:
# Ensure all the parameters are going to be updated regarding the transformer

In [14]:
for name,param in model.ray_transformer.named_parameters():
    if param.requires_grad:
        print(name)

ray_transformer.w_qs.weight
ray_transformer.w_ks.weight
ray_transformer.w_vs.weight
ray_transformer.fc.weight
ray_transformer.layer_norm.weight
ray_transformer.layer_norm.bias
out_alpha_linear.0.weight
out_alpha_linear.0.bias
out_alpha_linear.2.weight
out_alpha_linear.2.bias


#### 2. Main ray rendering algorithm. 

In [9]:
z = model.encode(ray_batch['src_img'])


from model.render_ray import sample_along_camera_ray,run_network
from utils.general import compute_cosine_simmilarity
from model.nerf import run_nerf_symm_local

rays_o = ray_batch['rays_o']    # [B,256,3]
rays_d = ray_batch['rays_d']    # [B,256,3]
z_near = ray_batch['z_near']
z_far = ray_batch['z_far']

noise = False 

#### 1. Get the 3D points ray, the viewdir and z sampled dist.
pts, viewdirs, z_vals = sample_along_camera_ray(rays_o=rays_o,  # pts : [B,256,64,3] - viewdirs : [B,256,3] - z_vals : [B,256,64]
                                                rays_d=rays_d,
                                                z_near=z_near,
                                                z_far=z_far,
                                                device=device,
                                                N_samples=nb_sampled_points_on_rays,
                                                lindisp=lindisp, 
                                                det=det)
#### 2. Get the NeRF weights according to the latent code z. 
nerf_fine_layers = model.hypernetwork_fine(z) 

### 3. Get the local feature, corresponding grid and their symetric counterpart.
ret_features = model.feature_net.index(pts, 
                                       ray_batch['src_pose'],
                                       ray_batch['intrinsics'], 
                                       ray_batch['image_size'], 
                                       noise)

f, f_s = ret_features['feat'], ret_features['feat_s']
grid,grid_s = ret_features['uv'],ret_features['uv_symm']

local_feature = torch.cat([f,f_s],-1)
### 4. Compute cosine simmilarity. 
cs = compute_cosine_simmilarity(f,f_s,args.cosine_mod['G'])
args.cosine_mod['cs']=cs

### 5. Define the NeRF models (one for each image in the batch since they all have a different latent code z). 
nerf_fine = lambda x: run_nerf_symm_local(x, 
                                          nerf_layers=nerf_fine_layers, 
                                          input_ch=model.input_ch,
                                          input_ch_views=model.input_ch_views,
                                          local_feature_ch=1024,
                                          cosine_mod = args.cosine_mod
                                         )

network = {'nerf':nerf_fine,
          'ray_transformer':model.ray_transformer}

print(f'Shape of pts: {pts.shape}')
raw_fine = run_network(pts,viewdirs,network,model.embed_fn,model.embeddirs_fn,local_feature,args.cosine_mod,ret_features)

Shape of pts: torch.Size([2, 128, 128, 3])


In [ ]:
128*128

In [2]:
import torch
torch.cuda.empty_cache() # PyTorch thing

In [3]:
!pip install numba 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 46.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
from numba import cuda
 
cuda.select_device(0) # choosing second GPU 
cuda.close()